In [15]:
import cv2
import numpy as np
import random
import math
import time
import cupy as cp

class VoronoiMosaicGPU:
    def __init__(self, input_image):
        self.input_image = cp.array(input_image)
        self.region_assignment = None
        self.output_image = None

    # Generate seeds for the Voronoi diagram
    def generate_seeds(self, num_seeds):
        h, w = self.input_image.shape[:2]
        seeds = cp.column_stack((
            cp.random.randint(0, w, num_seeds),
            cp.random.randint(0, h, num_seeds)
        ))
        return seeds

    # Generate the Voronoi diagram
    def voronoi(self, seeds):
        height, width = self.input_image.shape[:2]
        points = cp.mgrid[0:height, 0:width].transpose(1, 2, 0).reshape(-1, 2)

        num_seeds = seeds.shape[0]
        batch_size = 100  # Process 100 seeds at a time
        region_assignment = cp.zeros(points.shape[0], dtype=cp.int32)
        min_distances = cp.full(points.shape[0], cp.inf)

        for i in range(0, num_seeds, batch_size):
            batch_seeds = seeds[i:i + batch_size]
            distances = cp.sum((points[..., cp.newaxis, :] - batch_seeds) ** 2, axis=-1)
            batch_min_indices = cp.argmin(distances, axis=-1)
            batch_min_distances = cp.min(distances, axis=-1)

            # Update global region_assignment and min_distances
            mask = batch_min_distances < min_distances
            region_assignment[mask] = batch_min_indices[mask] + i
            min_distances[mask] = batch_min_distances[mask]

        self.region_assignment = region_assignment.reshape(height, width)

    # Assign colors to each region
    def region_color(self, num_seeds):
        region_colors = cp.zeros((num_seeds, 3), dtype=cp.uint8)
        for i in range(num_seeds):
            mask = self.region_assignment == i
            if cp.any(mask):
                pixels = self.input_image[mask]
                region_colors[i] = cp.mean(pixels, axis=0)
        return region_colors.get()

    # Generate the Voronoi mosaic
    def generate_mosaic(self, num_seeds):
        seeds = self.generate_seeds(num_seeds)
        print("Seeds generated")
        self.voronoi(seeds)
        print("Voronoi diagram generated")
        region_colors = self.region_color(num_seeds)
        print("Region colors generated")
        self.output_image = cp.asnumpy(region_colors)[cp.asnumpy(self.region_assignment)]
        return self.output_image

time_init = time.time()

# Read the image and move it to the GPU
image = cv2.imread('lena.png')

# Create Voronoi mosaic object
voronoi_mosaic = VoronoiMosaicGPU(image)

# Generate the mosaic with 1000 seeds
output_image = voronoi_mosaic.generate_mosaic(3000)

# Save the output image
cv2.imwrite('output_gpu.jpg', output_image)

print("Time taken with GPU: ", time.time() - time_init)


Seeds generated
Voronoi diagram generated
Region colors generated
Time taken with GPU:  10.014613151550293
